In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df['C'] = df['date'].apply(lambda date: COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)
df.dropna(subset=ema + physical + social + sleep, inplace=True)

sets_df = pd.read_parquet(sets_file, engine='pyarrow')

In [7]:
index = 51
model_rows = [sets_df.iloc[index]]

alphas = [5e-4]
hidden_layer_sizess = [(128, 8), (64,2)]
activations = ["relu"]

for activation in activations:
    for hidden_layer_sizes in hidden_layer_sizess:
        for alpha in alphas:
                #for model_row in model_rows:
                for index, model_row in sets_df.iterrows():

                    print(f"Modeling yreatment:{full_dictionary[model_row['treatment']]} on outcome:{full_dictionary[model_row['outcome']]}")
                    # people can't change their demographics.. except maybe OS :)
                    if model_row['treatment'].startswith('D'):
                        print(f"Skippnig demographic treatment {model_row['treatment']}")
                        continue

                    if model_row['outcome'] not in ["Y2", "Y3"]:
                        print("Only doing PHQ right now")
                        continue
                    wbm = WBNeuralNetModel(
                        data=df,
                        #gamma=gamma,
                        alpha=alpha,
                        activation='relu',
                        max_iter=4000,
                        hidden_layer_sizes=hidden_layer_sizes,
                        treatment=model_row['treatment'],
                        outcome=model_row['outcome'],
                        separating_set=model_row['sets'].tolist(),
                        name=f"row:{index}"
                        )

                    print(f"activation={activation}, alpha={alpha}, hidden_layers={list(hidden_layer_sizes)}, pre_rsq={wbm.pre_r_squared}, post_r_sq={wbm.post_r_squared}")

Modeling yreatment:race on outcome:pam
Skippnig demographic treatment D2
Modeling yreatment:cohort year on outcome:pam
Skippnig demographic treatment D4
Modeling yreatment:studying on outcome:pam
Only doing PHQ
Modeling yreatment:in house on outcome:pam
Only doing PHQ
Modeling yreatment:sports on outcome:pam
Only doing PHQ
Modeling yreatment:traveling on outcome:pam
Only doing PHQ
Modeling yreatment:time in social location on outcome:pam
Only doing PHQ
Modeling yreatment:visits on outcome:pam
Only doing PHQ
Modeling yreatment:duration unlocked phone in social locations on outcome:pam
Only doing PHQ
Modeling yreatment:gender on outcome:phq2_score
Skippnig demographic treatment D1
Modeling yreatment:excercise on outcome:phq2_score
Binarizing using median value (excercise median=1.10e+04)
Directory 20240527_213303_070296 created.
activation=relu, alpha=0.0005, hidden_layers=[128, 8], pre_rsq=(0.43421680904773197, 0.22234708630229894), post_r_sq=(0.6384208207293914, 0.18064339621375813)
Mo

In [8]:
results_df = aggregate_results_from_subfolders('.')
results_df.to_csv('results.csv')